spawning child processes?

spawn child process?


In [ ]:
var {exec} = require('child_process');
var path = require('path');
var importer = require('../Core');

var multiline = process.platform === 'win32' ? '^\n ' : '\\\n ';
var multicmd = process.platform === 'win32' ? ' ^\n&& ' : ' \\\n&& ';

// read cmd template and execute each line?
function bashToExec(code) {
    return code
        .split(/\n/ig)
        .map(l => '{EXEC} ' + l)
        .join('\n')
        .replace(/\\((\s*\n)+\{EXEC\})+\s*&&/ig, multicmd)
        .replace(/\\((\s*\n)+\{EXEC\})+\s*/ig, multiline);
}

function execCmd(script, options) {
    // TODO: fix current working directory matching project name in the cwd: option
    // add some run commands to the bash script
    return importer.runAllPromises(
        bashToExec(script).split(/\s*\{EXEC\}\s*/ig)
            .filter(r => r.trim() !== '')
            .map(r => (resolve, reject) => {
                console.log('spawning process ' + r)
                const ps = exec(r.replace(/\$[0-9]+/ig, i => process.argv[parseInt(i.substr(1))+1] || ''), Object.assign(options || {}, {maxBuffer: 1024 * 50000}),  (err, result) => {
                    if (err) {
                        return reject(err);
                    }
                    return resolve(result);
                });
                let errors = '';
                let data = '';
                ps.stderr.on('data', d => {
                    console.error(d);
                    data += d;
                });
                ps.stdout.on('data', d => {
                    console.log(d);
                    errors += d;
                });
                ps.on('exit', (code, signal) => {
                    if(code > 0) {
                        const err = new Error(`child process exited with code ${code} and signal ${signal}`);
                        err.data = data + errors;
                        reject(err);
                    }
                });
            })
    );
};
module.exports = execCmd;

//        "test": "concurrently --kill-others --success first \"mocha **/*.spec.js\" \"node ./zuora-eloqua-express-mock.js\" ",
 


test child process?

In [ ]:
$$.async();
describe(() => {
    
})
execCmd(`ps -a`)
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));



convert bash to one liner?

In [ ]:
// read cmd template and execute each line?
var multiline = process.platform === 'win32' ? '^\n ' : '\\\n ';
var multicmd = process.platform === 'win32' ? ' ^\n; ' : ' \\\n&& ';

function bashToOne(code) {
    return code
        .split(/\n/ig)
        .map(l => '{EXEC} ' + l)
        .join('\n')
        .replace(/\\((\s*\n)+\{EXEC\})+\s*&&/ig, multicmd)
        .replace(/\\((\s*\n)+\{EXEC\})+/ig, multiline)
        .split(/\s*\{EXEC\}\s*/ig)
        .filter(r => r.trim() !== '')
        .join(multicmd);
}
module.exports = bashToOne;
